# Parliamentary Speech Topic Modeling

Applies BERTopic with GMM clustering to discover topics, then uses GPT-4 to classify them into 23 policy categories.

**Input**: Processed data from data_preprocessing.ipynb  
**Output**: Same dataframes with added topic classification columns  
**Method**: Segment-level topic modeling with GPM + OpenAI classification

## Setup & Configuration

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.mixture import GaussianMixture
from bertopic import BERTopic
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()
pd.options.display.max_columns = None

# Policy categories for classification (full CAP descriptions)
POLICY_CATEGORIES = {
    "Education": "Issues related to educational policies, primary and secondary schools, student loans and education finance, the regulation of colleges and universities, school reforms, teachers, vocational training, evening schools, safety in schools, efforts to improve educational standards, and issues related to libraries, dictionaries, teaching material, research in education",
    "Technology": "Issues related to science and technology transfer and international science cooperation, research policy, government space programs and space exploration, telephones and telecommunication regulation, broadcast media (television, radio, newspapers, films), weather forecasting, geological surveys, computer industry, cyber security",
    "Health": "Issues related to health care, health care reforms, health insurance, drug industry, medical facilities, medical workers, disease prevention, treatment, and health promotion, drug and alcohol abuse, mental health, research in medicine, medical liability and unfair medical practices",
    "Environment": "Issues related to environmental policy, drinking water safety, all kinds of pollution (air, noise, soil), waste disposal, recycling, climate change, outdoor environmental hazards (e.g., asbestos), species and forest protection, marine and freshwater environment, hunting, regulation of laboratory or performance animals, land and water resource conservation, research in environmental technology",
    "Housing": "Issues related to housing, urban affairs and community development, housing market, property tax, spatial planning, rural development, location permits, construction inspection, illegal construction, industrial and commercial building issues, national housing policy, housing for low-income individuals, rental housing, housing for the elderly, e.g., nursing homes, housing for the homeless and efforts to reduce homelessness, research related to housing",
    "Labor": "Issues related to labor, employment, employment programs, employee benefits, pensions and retirement accounts, minimum wage, labor law, job training, labor unions, worker safety and protection, youth employment and seasonal workers",
    "Defense": "Issues related to defense policy, military intelligence, espionage, weapons, military personnel, reserve forces, military buildings, military courts, nuclear weapons, civil defense, including firefighters and mountain rescue services, homeland security, military aid or arms sales to other countries, prisoners of war and collateral damage to civilian populations, military nuclear and hazardous waste disposal and military environmental compliance, defense alliances and agreements, direct foreign military operations, claims against military, defense research",
    "Government Operations": "Issues related to general government operations, the work of multiple departments, public employees, postal services, nominations and appointments, national mints, medals, and commemorative coins, management of government property, government procurement and contractors, public scandal and impeachment, claims against the government, the state inspectorate and audit, anti-corruption policies, regulation of political campaigns, political advertising and voter registration, census and statistics collection by government; issues related to local government, capital city and municipalities, including decentralization; issues related to national holidays",
    "Social Welfare": "Issues related to social welfare policy, the Ministry of Social Affairs, social services, poverty assistance for low-income families and for the elderly, parental leave and child care, assistance for people with physical or mental disabilities, including early retirement pension, discounts on public services, volunteer associations (e.g., Red Cross), charities, and youth organizations",
    "Macroeconomics": "Issues related to domestic macroeconomic policy, such as the state and prospect of the national economy, economic policy, inflation, interest rates, monetary policy, cost of living, unemployment rate, national budget, public debt, price control, tax enforcement, industrial revitalization and growth",
    "Domestic Commerce": "Issues related to banking, finance and internal commerce, including stock exchange, investments, consumer finance, mortgages, credit cards, insurance availability and cost, accounting regulation, personal, commercial, and municipal bankruptcies, programs to promote small businesses, copyrights and patents, intellectual property, natural disaster preparedness and relief, consumer safety; regulation and promotion of tourism, sports, gambling, and personal fitness; domestic commerce research",
    "Civil Rights": "Issues related to civil rights and minority rights, discrimination towards races, gender, sexual orientation, handicap, and other minorities, voting rights, freedom of speech, religious freedoms, privacy rights, protection of personal data, abortion rights, anti-government activity groups (e.g., local insurgency groups), religion and the Church",
    "International Affairs": "Issues related to international affairs, foreign policy and relations to other countries, issues related to the Ministry of Foreign Affairs, foreign aid, international agreements (such as Kyoto agreement on the environment, the Schengen agreement), international organizations (including United Nations, UNESCO, International Olympic Committee, International Criminal Court), NGOs, issues related to diplomacy, embassies, citizens abroad; issues related to border control; issues related to international finance, including the World Bank and International Monetary Fund, the financial situation of the EU; issues related to a foreign country that do not impact the home country; issues related to human rights in other countries, international terrorism",
    "Transportation": "Issues related to mass transportation construction and regulation, bus transport, regulation related to motor vehicles, road construction, maintenance and safety, parking facilities, traffic accidents statistics, air travel, rail travel, rail freight, maritime transportation, inland waterways and channels, transportation research and development",
    "Immigration": "Issues related to immigration, refugees, and citizenship, integration issues, regulation of residence permits, asylum applications; criminal offences and diseases caused by immigration",
    "Law and Crime": "Issues related to the control, prevention, and impact of crime; all law enforcement agencies, including border and customs, police, court system, prison system; terrorism, white collar crime, counterfeiting and fraud, cyber-crime, drug trafficking, domestic violence, child welfare, family law, juvenile crime",
    "Agriculture": "Issues related to agriculture policy, fishing, agricultural foreign trade, food marketing, subsidies to farmers, food inspection and safety, animal and crop disease, pest control and pesticide regulation, welfare for animals in farms, pets, veterinary medicine, agricultural research",
    "Foreign Trade": "Issues related to foreign trade, trade negotiations, free trade agreements, import regulation, export promotion and regulation, subsidies, private business investment and corporate development, competitiveness, exchange rates, the strength of national currency in comparison to other currencies, foreign investment and sales of companies abroad",
    "Culture": "Issues related to cultural policies, Ministry of Culture, public spending on culture, cultural employees, issues related to support of theatres and artists; allocation of funds from the national lottery, issues related to cultural heritage",
    "Public Lands": "Issues related to national parks, memorials, historic sites, and protected areas, including the management and staffing of cultural sites; museums; use of public lands and forests, establishment and management of harbors and marinas; issues related to flood control, forest fires, livestock grazing",
    "Energy": "Issues related to energy policy, electricity, regulation of electrical utilities, nuclear energy and disposal of nuclear waste, natural gas and oil, drilling, oil spills, oil and gas prices, heat supply, shortages and gasoline regulation, coal production, alternative and renewable energy, energy conservation and energy efficiency, energy research",
    "Other": "Other topics not mentioning policy agendas, including the procedures of parliamentary meetings, e.g., points of order, voting procedures, meeting logistics; interpersonal speech, e.g., greetings, personal stories, tributes, interjections, arguments between the members; rhetorical speech, e.g., jokes, literary references",
    "Mix": "Use this category when the topic clearly spans multiple policy areas or when there is significant uncertainty about which single category best fits the topic. This is for topics that genuinely combine elements from 2-3 different categories in a meaningful way, making it difficult to assign to just one category with high confidence"
}

# Language-specific stopwords (comprehensive lists)
ENGLISH_STOPWORDS = [
    'mr', 'mrs', 'ms', 'dr', 'madam', 'honorable', 'honourable', 'member', 'members', 'vp', 'sp', 'fp', 'ae', 'po',
    'minister', 'speaker', 'deputy', 'president', 'chairman', 'chair', 'schilling', 'my', 'lords', 'lord', 'bzs', 'prll', 'bz',
    'secretary', 'lord', 'gp', 'lady', 'question', 'order', 'point', 'debate', 'motion', 'amendment', 'backbench', 'week',
    'congratulations', 'congratulate', 'thanks', 'thank', 'say', 'one', 'want', 'know', 'think', 'noble', 'opg',
    'believe', 'see', 'go', 'come', 'give', 'take', 'people', 'federal', 'government', 'austria', 'baroness',
    'austrian', 'committee', 'call', 'said', 'already', 'please', 'request', 'proceed', 'reading', 'prime',
    'course', 'welcome', 'council', 'open', 'written', 'contain', 'items', 'item', 'yes', 'no',
    'following', 'next', 'speech', 'year', 'years', 'state', 'also', 'would', 'like', 'may', 'must',
    'upon', 'indeed', 'session', 'meeting', 'report', 'commission', 'behalf', 'gentleman', 'gentlemen',
    'ladies', 'applause', 'group', 'colleague', 'colleagues', 'issue', 'issues', 'chancellor', 'court',
    'ask', 'answer', 'reply', 'regard', 'regarding', 'regards', 'respect', 'respectfully', 'sign',
    'shall', 'procedure', 'declare', 'hear', 'minutes', 'speaking', 'close', 'abg', 'mag', 'orf', 'wait'
]

GERMAN_STOPWORDS = [
    'der', 'die', 'das', 'und', 'in', 'zu', 'den', 'mit', 'von', 'für', 'bb', 'bz', 'bzs', 'prll',
    'auf', 'ist', 'im', 'sich', 'eine', 'sie', 'dem', 'nicht', 'ein', 'als',
    'auch', 'es', 'an', 'werden', 'aus', 'er', 'hat', 'dass', 'wir', 'ich',
    'haben', 'sind', 'kann', 'sehr', 'meine', 'muss', 'doch', 'wenn', 'sein',
    'dann', 'weil', 'bei', 'nach', 'so', 'oder', 'aber', 'vor', 'über', 'noch',
    'nur', 'wie', 'war', 'waren', 'wird', 'wurde', 'wurden', 'ihr', 'ihre',
    'ihren', 'seiner', 'seine', 'seinem', 'seinen', 'dieser', 'diese', 'dieses',
    'durch', 'ohne', 'gegen', 'unter', 'zwischen', 'während', 'bis', 'seit',
    'danke', 'bitte', 'gern', 'abgeordnete', 'abgeordneten', 'bundesregierung',
    'bundeskanzler', 'nationalrat', 'bundesrat', 'parlament', 'fraktion',
    'ausschuss', 'sitzung', 'präsident', 'vizepräsident', 'minister',
    'staatssekretär', 'klubobmann', 'antrag', 'anfrage', 'interpellation',
    'dringliche', 'aktuelle', 'stunde', 'debatte', 'abstimmung', 'beschluss',
    'gesetz', 'novelle', 'verordnung', 'regierungsvorlage', 'initiativantrag',
    'danke', 'dankeschön', 'geschätzte', 'kolleginnen', 'kollegen', 'hohes'
]

CROATIAN_STOPWORDS = [
    'a', 'ako', 'ali', 'bi', 'bih', 'bila', 'bili', 'bilo', 'bio', 'bismo',
    'biste', 'biti', 'bumo', 'da', 'do', 'duž', 'ga', 'hoće', 'hoćemo',
    'hoćete', 'hoćeš', 'hoću', 'i', 'iako', 'ih', 'ili', 'iz', 'ja', 'je',
    'jedna', 'jedne', 'jedno', 'jer', 'jesam', 'jesi', 'jesmo', 'jest',
    'jeste', 'jesu', 'jim', 'joj', 'još', 'ju', 'kada', 'kako', 'kao',
    'koja', 'koje', 'koji', 'kojima', 'koju', 'kroz', 'li', 'me', 'mene',
    'meni', 'mi', 'mimo', 'moj', 'moja', 'moje', 'mu', 'na', 'nad', 'nakon',
    'nam', 'nama', 'nas', 'naš', 'naša', 'naše', 'našeg', 'ne', 'nego',
    'neka', 'neki', 'nekog', 'neku', 'nema', 'netko', 'neće', 'nećemo',
    'nećete', 'nećeš', 'neću', 'nešto', 'ni', 'nije', 'nikoga', 'nikoje',
    'nikoju', 'nisam', 'nisi', 'nismo', 'niste', 'nisu', 'njega', 'njegov',
    'njegova', 'njegovo', 'njemu', 'njezin', 'njezina', 'njezino', 'njih',
    'njihov', 'njihova', 'njihovo', 'njim', 'njima', 'njoj', 'nju', 'no',
    'o', 'od', 'odmah', 'on', 'ona', 'oni', 'ono', 'ova', 'pa', 'pak',
    'po', 'pod', 'pored', 'prije', 's', 'sa', 'sam', 'samo', 'se', 'sebe',
    'sebi', 'si', 'smo', 'ste', 'su', 'sve', 'svi', 'svog', 'svoj', 'svoja',
    'svoje', 'svom', 'ta', 'tada', 'taj', 'tako', 'te', 'tebe', 'tebi',
    'ti', 'to', 'toj', 'tome', 'tu', 'tvoj', 'tvoja', 'tvoje', 'u', 'uz',
    'vam', 'vama', 'vas', 'vaš', 'vaša', 'vaše', 'već', 'vi', 'vrlo', 'za',
    'zar', 'će', 'ćemo', 'ćete', 'ćeš', 'ću', 'što', 'zastupnik', 'zastupnica',
    'zastupnici', 'hvala', 'sabor', 'hrvatska', 'vlada', 'molim', 'gospodin',
    'gospođa', 'premijer', 'predsjednik', 'predsjednica', 'ministar', 'ministrica',
    'državni', 'tajnik', 'tajnica', 'odbor', 'sjednica', 'rasprava', 'prijedlog',
    'zakon', 'odluka', 'glasovanje', 'amandman', 'interpelacija', 'pitanje',
    'odgovor', 'klupski', 'obnašatelj', 'dužnosti', 'potpredsjednik',
    'potpredsjednica', 'kolegice', 'kolege', 'dame', 'gospodo', 'poštovani', 'poštovana'
]

STOPWORDS = {
    "english": ENGLISH_STOPWORDS,
    "german": GERMAN_STOPWORDS,
    "croatian": CROATIAN_STOPWORDS
}

print("✅ Configuration loaded")
print(f"   Policy categories: {len(POLICY_CATEGORIES)}")
print(f"   Stopwords: EN={len(ENGLISH_STOPWORDS)}, DE={len(GERMAN_STOPWORDS)}, HR={len(CROATIAN_STOPWORDS)}")

## Load Data

Load the processed dataframes from data_preprocessing.ipynb (from OUTPUT_DIR/processed folder).

In [ ]:
import os

# Path where data_preprocessing.ipynb saves processed data
BASE_DATA_DIR = r"data folder"
PROCESSED_DIR = os.path.join(BASE_DATA_DIR, "processed")

# Load processed datasets
AT = pd.read_pickle(os.path.join(PROCESSED_DIR, "AT_speeches_processed.pkl"))
HR = pd.read_pickle(os.path.join(PROCESSED_DIR, "HR_speeches_processed.pkl"))
GB = pd.read_pickle(os.path.join(PROCESSED_DIR, "GB_speeches_processed.pkl"))

print(f"✅ Loaded from: {PROCESSED_DIR}")
print(f"   AT={AT.shape}, HR={HR.shape}, GB={GB.shape}")

## Topic Modeling Functions

In [ ]:
class GMMClustering:
    """GMM clustering for BERTopic"""
    def __init__(self, n_components=200, random_state=42):
        self.n_components = n_components
        self.random_state = random_state
        self.labels_ = None
    
    def fit(self, X, y=None):
        model = GaussianMixture(n_components=self.n_components, random_state=self.random_state,
                               covariance_type='tied', max_iter=300)
        model.fit(X)
        self.labels_ = model.predict(X)
        return self
    
    def fit_predict(self, X):
        self.fit(X)
        return self.labels_


def create_topic_model(language, n_clusters=200):
    """Create BERTopic model with GMM clustering"""
    vectorizer = CountVectorizer(
        stop_words=STOPWORDS.get(language, STOPWORDS['english']),
        ngram_range=(1, 2), min_df=5, max_df=0.9, max_features=20000
    )
    
    umap_model = UMAP(n_neighbors=15, n_components=10, min_dist=0.05, 
                     metric='cosine', random_state=42)
    
    gmm_model = GMMClustering(n_components=n_clusters)
    
    return BERTopic(
        vectorizer_model=vectorizer,
        umap_model=umap_model,
        hdbscan_model=gmm_model,
        embedding_model=None,
        top_n_words=15,
        verbose=True
    )


def prepare_segments(df, segment_col, text_col, embedding_col):
    """Group speeches into segments"""
    grouped = df.groupby(segment_col).agg({
        text_col: ' '.join,
        embedding_col: 'first'
    }).reset_index()
    
    return (grouped[text_col].tolist(), 
            np.array(grouped[embedding_col].tolist()),
            grouped[segment_col].tolist())


def classify_with_gpt(topic_words, country, language):
    """Classify topic using GPT-4"""
    categories_str = '\n'.join([f"• {cat}: {desc}" for cat, desc in POLICY_CATEGORIES.items()])
    
    prompt = f"""Classify these parliamentary keywords into ONE policy category.
    
Country: {country} Parliament
Language: {language}
Keywords: {', '.join(topic_words)}

Categories:
{categories_str}

Instructions:
- Choose the most specific policy category
- Use "Other" for procedural/non-policy content
- Use "Mix" only if clearly spanning multiple domains
- Be conservative: default to "Other" if uncertain

Format: CATEGORY: [exact category name]"""
    
    try:
        client = OpenAI()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a parliamentary policy classifier."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.01,
            max_tokens=300
        )
        
        text = response.choices[0].message.content.strip()
        for line in text.split('\n'):
            if line.startswith('CATEGORY:'):
                category = line.split(':', 1)[1].strip().replace('"', '').replace("'", "")
                if category in POLICY_CATEGORIES:
                    return category
        return "Other"
    except Exception as e:
        print(f"Error: {e}")
        return "Other"


def run_topic_pipeline(df, country, language, text_col, segment_col, embedding_col, n_clusters=200):
    """Complete topic modeling pipeline"""
    print(f"\n{'='*60}")
    print(f"{country} Parliament - {language}")
    print(f"{'='*60}")
    
    # Prepare data
    documents, embeddings, segment_ids = prepare_segments(df, segment_col, text_col, embedding_col)
    print(f"Processing {len(documents)} segments...")
    
    # Fit model
    topic_model = create_topic_model(language, n_clusters)
    topics, _ = topic_model.fit_transform(documents, embeddings.astype(np.float32))
    topic_info = topic_model.get_topic_info()
    
    print(f"Discovered {len(set(topics))} topics")
    
    # Classify topics
    print("Classifying with GPT-4...")
    topic_categories = {}
    for idx, row in tqdm(topic_info.iterrows(), total=len(topic_info)):
        topic_id = row['Topic']
        words = [w for w, _ in topic_model.get_topic(topic_id)]
        category = classify_with_gpt(words, country, language)
        topic_categories[topic_id] = category
        time.sleep(0.3)
    
    # Map back to dataframe
    segment_categories = [topic_categories.get(t, "Other") for t in topics]
    segment_df = pd.DataFrame({
        segment_col: segment_ids,
        f'Topic_{country}_{language}': segment_categories
    })
    
    df_result = df.merge(segment_df, on=segment_col, how='left')
    
    # Show distribution
    cat_dist = pd.Series(segment_categories).value_counts()
    print(f"\nTop categories:")
    for cat, count in cat_dist.head(5).items():
        print(f"  {cat}: {count}")
    
    return df_result, topic_categories

print("✅ Functions defined")

## Apply Topic Modeling

Run topic modeling on all datasets and languages.

In [ ]:
# GB (English only)
GB_final, gb_cats = run_topic_pipeline(
    GB, 'GB', 'english', 'Text', 'Segment_ID', 
    'Segment_Embeddings_Text', n_clusters=200
)

# AT (English + German)
AT_temp, at_en_cats = run_topic_pipeline(
    AT, 'AT', 'english', 'Text', 'Segment_ID_english',
    'Segment_Embeddings_Text', n_clusters=200
)

AT_final, at_de_cats = run_topic_pipeline(
    AT_temp, 'AT', 'german', 'Text_native_language', 'Segment_ID_native',
    'Segment_Embeddings_Text_native_language', n_clusters=200
)

# HR (English + Croatian)
HR_temp, hr_en_cats = run_topic_pipeline(
    HR, 'HR', 'english', 'Text', 'Segment_ID_english',
    'Segment_Embeddings_Text', n_clusters=200
)

HR_final, hr_hr_cats = run_topic_pipeline(
    HR_temp, 'HR', 'croatian', 'Text_native_language', 'Segment_ID_native',
    'Segment_Embeddings_Text_native_language', n_clusters=200
)

print("\n✅ Topic modeling complete for all datasets")

## Save Results

In [ ]:
# Save final dataframes back to processed folder
GB_final.to_pickle(os.path.join(PROCESSED_DIR, "GB_with_topics.pkl"))
AT_final.to_pickle(os.path.join(PROCESSED_DIR, "AT_with_topics.pkl"))
HR_final.to_pickle(os.path.join(PROCESSED_DIR, "HR_with_topics.pkl"))

print("✅ Saved all results to:", PROCESSED_DIR)
print(f"\nFinal columns added:")
print(f"  GB: Topic_GB_english")
print(f"  AT: Topic_AT_english, Topic_AT_german")
print(f"  HR: Topic_HR_english, Topic_HR_croatian")

## Summary

View topic distribution across all datasets.

In [ ]:
# Combine all topic columns for overview
all_topics = []
for col in ['Topic_GB_english']:
    all_topics.extend(GB_final[col].dropna().tolist())
for col in ['Topic_AT_english', 'Topic_AT_german']:
    all_topics.extend(AT_final[col].dropna().tolist())
for col in ['Topic_HR_english', 'Topic_HR_croatian']:
    all_topics.extend(HR_final[col].dropna().tolist())

topic_dist = pd.Series(all_topics).value_counts()

print("📊 Overall Topic Distribution")
print("="*40)
for i, (topic, count) in enumerate(topic_dist.head(10).items(), 1):
    pct = count / len(all_topics) * 100
    print(f"{i:2d}. {topic}: {count:,} ({pct:.1f}%)")

print(f"\nTotal: {len(all_topics):,} classified segments")
print("✅ Topic modeling pipeline complete!")